# EEG - Flow

## 5. ICA compare components selections

Last edit: 07.05.2023 21:09
@anguyen

In [ ]:
import os
from datetime import datetime
import matplotlib.pyplot as plt

from mne.viz import set_browser_backend

%matplotlib qt
set_browser_backend('qt')

from eeg_flow.tasks.ica_compare import(
    prep_ica_compare,
    load_ica_rev,
    compare_two_revs,
)

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
"""
PARTICIPANT: str (for example "P03", "P28")
GROUP:       str ["G1", "G2", "G3", "G4", "G5", "G6", "G7", "G8"]
TASK:        str, ["oddball", "UT"]
RUN:         int [1, 2]
ICA_NB:      int [1, 2]
"""

PARTICIPANT = "P02"
GROUP       = "G2"
TASK        = "oddball"
RUN         = 1
ICA_NB      = 1

REVIEWER1 = "anguyen3"
REVIEWER2 = "anguyen4"

In [ ]:
DERIVATIVES_SUBFOLDER, FNAME_STEM, raw, raw_ica_fit, locks = prep_ica_compare(PARTICIPANT, GROUP, TASK, RUN, ICA_NB)
DERIVATIVES_ICA = DERIVATIVES_SUBFOLDER / "plots" / "ica"

In [ ]:
ica_rev1, ica_rev2 = load_ica_rev(
    DERIVATIVES_SUBFOLDER, FNAME_STEM, REVIEWER1, REVIEWER2, ICA_NB
)

In [ ]:
exclude_common, exclude_diff = compare_two_revs(ica_rev1, ica_rev2)

In [ ]:
# create figure to validate components that have been commonly excluded.
if len(exclude_common) != 0:
    f_common = ica1_rev1.plot_components(picks=exclude_common, inst=raw_ica_fit, show=False)
    f_common.show()


In [ ]:
print(exclude_common)
print(exclude_diff)

In [ ]:
%matplotlib inline
# to avoid the bug
plt.figure(figsize=(2,2))
plt.plot([])

In [ ]:
# this is to bypass the fact that ica plot components with picks not none only
# plots the first 20 components

def divide_chunks(exclude_diff):
    # looping till length l
    for i in range(0, len(exclude_diff),20):
        yield exclude_diff[i:i + 20]

exclude_diff_per20 = list(divide_chunks(exclude_diff))

In [ ]:
# create figure to validate components that have been differently excluded.
%matplotlib qt
print(len(exclude_diff_per20))
f_dif = []
if len(exclude_diff) != 0:
    for chunk in exclude_diff_per20:
        f_dif.append(ica_rev1.plot_components(
            title=f"{FNAME_STEM} | ICA{ICA_NB} | Reviewers difference",
            picks=chunk,
            inst=raw_ica_fit,
            show=True)
            )

In [ ]:
figs_ica_sources = ica_rev1.plot_sources(
    title=f"{FNAME_STEM} | ICA{ICA_NB}",
    show=True,
    inst=raw_ica_fit,
)

In [ ]:
timestampStr = datetime.now().strftime("%Y-%m-%d__%H-%M")
for i, fig in enumerate(f_dif):
    FNAME_ICA_DIFF = (
        DERIVATIVES_ICA / f"allComponents_ICA{ICA_NB}_revdiff_{i+1}_timestampStr.svg"
    )
    fig.savefig(FNAME_ICA_DIFF, transparent=True)

In [ ]:
#plot final res

In [ ]:
# same here, this might require to be run twice
%matplotlib qt
figs_ica_comp = ica_rev1.plot_components(
    title=f"{FNAME_STEM} | ICA{ICA_NB} Final review",
    show=True,
    inst=raw_ica_fit
)

In [ ]:
timestampStr = datetime.now().strftime("%Y-%m-%d__%H-%M")
for i, fig in enumerate(figs_ica_comp):
    FNAME_ICA_COMP = (
        DERIVATIVES_ICA / f"allComponents_ICA{ICA_NB}_Final{i+1}_timestampStr.svg"
    )
    fig.savefig(FNAME_ICA_COMP, transparent=True)

## 4.1 Annotate bad ICs from ICA1 for mastoids
 - At this stage, let's only focus on the mastoids. Look for:
 - heartbeat in the IC-time series
 - muscle/noise on the mastoids on the topographic map

## 4.3 Save derivatives

The ICA decomposition can be saved.

In [ ]:
if ICA_NB == 1:
    FNAME_ICA = DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step5_reviewed-1st-ica.fif")
elif ICA_NB == 2:
    FNAME_ICA = DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step5_reviewed-2nd-ica.fif")

ica_rev1.save(FNAME_ICA, overwrite=False)


Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway

Output of step 5
- fitted ICA (_reviewed-{1st or 2nd}-ica.fif)